In [73]:
!pip install requests
import requests
import pandas as pd
import csv

In [74]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
#response.text

In [75]:
!pip install beautifulsoup4
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
#response.text

soup = BeautifulSoup(response.text, "html.parser")

In [76]:
table = soup.find_all('table', class_ = 'wikitable sortable' )
#print(table)

#### Scrape table and convert it into a datafram

In [77]:
table = soup.find_all('table')
df = pd.read_html(str(table))[0]
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


#### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [78]:
df = df[df.Borough != 'Not assigned']
df = df.rename(columns={'Postal code': 'PostalCode'})
df.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don Mills
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


#### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [79]:
#because the website already aggregated the data they added a / into it. I need to replace it with a , 
for col in df:
    df['Neighborhood'] = df['Neighborhood'].str.replace(' /',', ')
    
df.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


#### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [80]:
df.shape

(103, 3)

### Part 2 Get the latitude and the longitude coordinates of each neighborhood.

In [81]:
!pip install geocoder
import geocoder # import geocoder

#### Import postal code files

In [82]:
dfpostalcodes = pd.read_csv('https://cocl.us/Geospatial_data')
print(dfpostalcodes)

    Postal Code   Latitude  Longitude
0           M1B  43.806686 -79.194353
1           M1C  43.784535 -79.160497
2           M1E  43.763573 -79.188711
3           M1G  43.770992 -79.216917
4           M1H  43.773136 -79.239476
..          ...        ...        ...
98          M9N  43.706876 -79.518188
99          M9P  43.696319 -79.532242
100         M9R  43.688905 -79.554724
101         M9V  43.739416 -79.588437
102         M9W  43.706748 -79.594054

[103 rows x 3 columns]


#### get postal code that is in df


In [84]:
postcodes = df['PostalCode'].values
lat = []
lon = []

for pc in postcodes:
        found = dfpostalcodes[dfpostalcodes['Postal Code'] == pc]
        lat.append(found.Latitude.iloc[0])
        lon.append(found.Longitude.iloc[0])

df['Latitude'] = lat
df['Longitude']= lon

df.head(30)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
8,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
9,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
11,M3B,North York,Don Mills,43.745906,-79.352188
12,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
